# Randomized Cholesky QR


While [CholeskyQR](./cholesky-qr.ipynb) has a high flop efficiency, it is unstable due to forming the Gram matrix $\vec{A}^\T\vec{A}$.


## Subspace Embedding for approximate QR


```{prf:algorithm} Randomized approximate QR
:label: sketched-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Sample $\vec{S}\sim\Call{Sketch}(k,n)$
1. For $\vec{Y} = \vec{S}\vec{A}$
1. Compute QR factorization $\vec{Q}'\vec{R} = \Call{qr}(\vec{Y})$.
1. Form $\vec{Q} = \vec{A}\vec{R}^{-1}$

**Output:** $\vec{Q}, \vec{R}$
```


````{prf:theorem} Well-conditioned basis from subspace embedding

Suppose $\vec{A}$ is full-ranka and $\vec{S}$ is an $\varepsilon$-subspace embedding for $\vec{A}$.
Then the output $\vec{Q}$ of {prf:ref}`sketched-qr` is well-conditioned in the sense that 
```{math}
\frac{1}{1+\varepsilon} \leq \smin(\vec{Q}) \leq \smax(\vec{Q}) \leq \frac{1}{1-\varepsilon}.
```
````


````{admonition} Proof
:class: dropdown

By definition, 
```{math}
\forall \vec{x}\in\range(\vec{A}): (1-\varepsilon)\|\vec{x}\|_2 \leq \|\vec{S}\vec{x}\|_2 \leq (1+\varepsilon)\|\vec{x}\|_2.
```
We can reparameterize $\vec{x} = \vec{A}\vec{R}^{-1}\vec{c} = \vec{Q}\vec{c}$ for some $\vec{c}\in\R^d$.
Since $\vec{S}\vec{A}\vec{R}^{-1} = \vec{Q}'$ has orthonormal columns, we have $\|\vec{S}\vec{x}\| = \|\vec{S}\vec{A}\vec{R}^{-1}\vec{c}\|_2 = \|\vec{c}\|_2$.
Thus,
```{math}
(1-\varepsilon) \|\vec{Q}\vec{c}\|_2 \leq \|\vec{c}\|_2 \leq (1+\varepsilon)\|\vec{Q}\vec{c}\|_2.
```
Hence, 
```{math}
\smin(\vec{Q}) = \min_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \geq \frac{1}{1+\varepsilon} 
,\qquad
\smax(\vec{Q}) = \max_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \leq \frac{1}{1-\varepsilon}.
````


Unfortuatnely, [oblivious sketching methods](../Background/sketching.ipynb) require $k$ to grow polynomially with $\varepsilon$. 
As such, {prf:ref}`sketched-qr` is not suitable for providing a nearly orthonormal basis.


## Preconditioning for QR


{prf:alg}`sketched-qr` can efficiently provide a well-conditioned basis for $\vec{A}$. 



```{prf:algorithm} Randomized Cholesky QR
:label: randomized-cholesky-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Compute approximate QR factorization $\vec{Q}_1,\vec{R}_1 = \Call{rand-qr}(\vec{A})$
1. $\vec{Q},\vec{R}_1 = \Call{cholesky-qr}(\vec{Q}_1)$
1. $\vec{R} = \vec{R}_2\vec{R}_1$

**Output:** $\vec{Q}, \vec{R}$
```

## Numerical Experiment

Let's try to understand the performance of Randomized Cholesky QR relative to 